In [3]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist, cifar10

In [4]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")



In [5]:

class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)


In [6]:

class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units

    def call(self, inputs):
        return tf.matmul(inputs, self.W) + self.b

    def build(self, input_shape):

        self.W = self.add_weight(
            name='W',
            shape=(input_shape[-1], self.units),
            initializer='random_normal',
            trainable=True
        )
        self.b = self.add_weight(
            name='b',
            shape=(self.units,),
            initializer='zeros',
            trainable=True
        )

In [7]:
class MyModel(keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()

        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)

    def call(self, input_tensor):
        # x = tf.nn.relu(self.dense1(input_tensor))
        x = self.relu(self.dense1(input_tensor))
        return self.dense2(x)



In [8]:
import pywt
@tf.autograph.experimental.do_not_convert
def dwt(x):
    coeffs2 = pywt.dwt2(x, 'bior1.3')
    LL, (LH, HL, HH) = coeffs2
    tf.convert_to_tensor([LL, LH, HL, HH])
    return x

In [9]:
class MyDWTModel(keras.Model):
    def __init__(self):
        super(MyDWTModel, self).__init__()

        self.conv1 = layers.Conv2D(32, 1)
        self.pooling2d = layers.MaxPooling2D(pool_size=2)
        self.bn1 = layers.BatchNormalization()
        self.relu = MyReLU()
        
        self.flat = layers.Flatten()
        self.dense = layers.Dense(32)


    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        #x = dwt(x.numpy())
        x = self.pooling2d(x)
        x = self.relu(x)
        x = self.flat(x)
        x = self.dense(x)

        return x




In [10]:

model = MyDWTModel()

a = model((32,32,3))

ValueError: Layer conv2d expects 1 inputs, but it received 3 input tensors. Inputs received: [<tf.Tensor: shape=(), dtype=int32, numpy=32>, <tf.Tensor: shape=(), dtype=int32, numpy=32>, <tf.Tensor: shape=(), dtype=int32, numpy=3>]

In [ ]:
model.layers

In [ ]:
# model.variables

In [ ]:
# 

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

In [ ]:
model.fit(x_train, y_train, batch_size=32, verbose=2)

In [ ]:
model.summary()